In [194]:
import pandas as pd
import numpy as np
import datetime

In [195]:
#Step 1: Import document, Change Program Start Date to Date of Identification
df = pd.read_csv("February.csv")
df = df.rename(columns={"Program Start Date":"Date of Identification",'Case Number':'Client ID','Veteran Status (HUD)':'Veteran Status'})
df = df.drop(["Relationship","Family Name",'Name'],axis=1)
df = df.drop_duplicates()
df = df.reset_index()
df = df.drop(['index'],axis=1)
# df.index.name = 'Index'
df.head()

,Client ID,Program Name,Veteran Status,Date of Identification,Program End Date,Dismissal Reason
0,903,Arlington Zero: Chronic - Veterans Only,No,2019/07/01 00:00:00,NaN,NaN
1,3734,Arlington Zero: Single Adults,No,2021/04/09 00:00:00,NaN,NaN
2,36862,Arlington Zero: Single Adults,No,2021/06/30 00:00:00,2022/02/04 00:00:00,Exit to Permanent Housing
3,42796,Arlington Zero: Single Adults,No,2021/09/08 00:00:00,NaN,NaN
4,51829,Arlington Zero: Chronic - Veterans Only,Yes,2021/12/10 00:00:00,NaN,NaN


In [196]:
#Step 2: Adding Household Type Column
household_type = {"Arlington Zero: Chronic - Veterans Only":"Single Adults",
"Arlington Zero: Single Adults":"Single Adults",
"Arlington Zero: Families":"Families",
"Arlington Zero: TAY":"Youth"}
df['Household Type'] = df['Program Name'].map(household_type)
df.head()

,Client ID,Program Name,Veteran Status,Date of Identification,Program End Date,Dismissal Reason,Household Type
0,903,Arlington Zero: Chronic - Veterans Only,No,2019/07/01 00:00:00,NaN,NaN,Single Adults
1,3734,Arlington Zero: Single Adults,No,2021/04/09 00:00:00,NaN,NaN,Single Adults
2,36862,Arlington Zero: Single Adults,No,2021/06/30 00:00:00,2022/02/04 00:00:00,Exit to Permanent Housing,Single Adults
3,42796,Arlington Zero: Single Adults,No,2021/09/08 00:00:00,NaN,NaN,Single Adults
4,51829,Arlington Zero: Chronic - Veterans Only,Yes,2021/12/10 00:00:00,NaN,NaN,Single Adults


In [197]:
#Step 3: Adding Client ID Counter and Client ID Household Counter
#Client ID Counter
df['Client ID Counter'] = df['Client ID'].map(df.groupby('Client ID').agg({'Client ID':'count'})['Client ID'])
#Client ID Household Counter
counter = {}
for i in df['Client ID']:
    if i not in counter:
        counter[i] = {"Single Adults":0,"Families":0,"Youth":0}
for j in counter:
    for k in df[df['Client ID']==j]['Household Type']:
        counter[j][k] += 1
df['Client ID Household Counter'] = np.nan
for i in df.index:
    df.loc[i,'Client ID Household Counter'] = counter[df.loc[i,'Client ID']][df.loc[i,'Household Type']]
df.head()

,Client ID,Program Name,Veteran Status,Date of Identification,Program End Date,Dismissal Reason,Household Type,Client ID Counter,Client ID Household Counter
0,903,Arlington Zero: Chronic - Veterans Only,No,2019/07/01 00:00:00,NaN,NaN,Single Adults,1,1.0
1,3734,Arlington Zero: Single Adults,No,2021/04/09 00:00:00,NaN,NaN,Single Adults,1,1.0
2,36862,Arlington Zero: Single Adults,No,2021/06/30 00:00:00,2022/02/04 00:00:00,Exit to Permanent Housing,Single Adults,1,1.0
3,42796,Arlington Zero: Single Adults,No,2021/09/08 00:00:00,NaN,NaN,Single Adults,1,1.0
4,51829,Arlington Zero: Chronic - Veterans Only,Yes,2021/12/10 00:00:00,NaN,NaN,Single Adults,1,1.0


In [198]:
#Step 4: Adding Chronic Column
#If client is in Chronic & Vet and 'No' to Veteran Status, then they are chronic
df['Chronic Status'] = np.nan
df.loc[(df['Program Name']=="Arlington Zero: Chronic - Veterans Only") & (df['Veteran Status']=="No")
,"Chronic Status"] = "Yes"
df['Chronic Status'].value_counts()

Yes    8
Name: Chronic Status, dtype: int64

In [199]:
#Step 5, remap all dismissal reasons
from values import dismissal
df['Dismissal Reason'] = df['Dismissal Reason'].map(dismissal)
df[df['Client ID']==2327]

,Client ID,Program Name,Veteran Status,Date of Identification,Program End Date,Dismissal Reason,Household Type,Client ID Counter,Client ID Household Counter,Chronic Status
18,2327,Arlington Zero: Chronic - Veterans Only,No,2021/06/21 00:00:00,2022/02/09 00:00:00,No longer meets population criteria,Single Adults,1,1.0,Yes


In [200]:
#Step 6, Populate Housing Move-In Date
df['Housing Move-In Date'] = df["Program End Date"][df["Dismissal Reason"]=="Housed"]
df.head()

,Client ID,Program Name,Veteran Status,Date of Identification,Program End Date,Dismissal Reason,Household Type,Client ID Counter,Client ID Household Counter,Chronic Status,Housing Move-In Date
0,903,Arlington Zero: Chronic - Veterans Only,No,2019/07/01 00:00:00,NaN,NaN,Single Adults,1,1.0,Yes,NaN
1,3734,Arlington Zero: Single Adults,No,2021/04/09 00:00:00,NaN,NaN,Single Adults,1,1.0,NaN,NaN
2,36862,Arlington Zero: Single Adults,No,2021/06/30 00:00:00,2022/02/04 00:00:00,Housed,Single Adults,1,1.0,NaN,2022/02/04 00:00:00
3,42796,Arlington Zero: Single Adults,No,2021/09/08 00:00:00,NaN,NaN,Single Adults,1,1.0,NaN,NaN
4,51829,Arlington Zero: Chronic - Veterans Only,Yes,2021/12/10 00:00:00,NaN,NaN,Single Adults,1,1.0,NaN,NaN


In [201]:
#Step 7, Populate Inactive Date
#Do we consider those with program end date and null dismissal reasons as inactive?
df['Inactive Date'] = df['Program End Date'][df["Dismissal Reason"]=="Inactive"]
df.head()

,Client ID,Program Name,Veteran Status,Date of Identification,Program End Date,Dismissal Reason,Household Type,Client ID Counter,Client ID Household Counter,Chronic Status,Housing Move-In Date,Inactive Date
0,903,Arlington Zero: Chronic - Veterans Only,No,2019/07/01 00:00:00,NaN,NaN,Single Adults,1,1.0,Yes,NaN,NaN
1,3734,Arlington Zero: Single Adults,No,2021/04/09 00:00:00,NaN,NaN,Single Adults,1,1.0,NaN,NaN,NaN
2,36862,Arlington Zero: Single Adults,No,2021/06/30 00:00:00,2022/02/04 00:00:00,Housed,Single Adults,1,1.0,NaN,2022/02/04 00:00:00,NaN
3,42796,Arlington Zero: Single Adults,No,2021/09/08 00:00:00,NaN,NaN,Single Adults,1,1.0,NaN,NaN,NaN
4,51829,Arlington Zero: Chronic - Veterans Only,Yes,2021/12/10 00:00:00,NaN,NaN,Single Adults,1,1.0,NaN,NaN,NaN


In [202]:
#Step 8, Calculate 1stDateofID, then calculate Returned to Active Date (Date of Idenfication on second record)
#{FIXED [ClientID],[Household Type]: MIN([Date of Identification]) }
#IF [NumRows_ClientID_HouseholdType]> 1 AND [1stDateofID] <> [Date of Identification] THEN [Date of Identification] ELSE NULL END
#If [Household Type Case Counter]>1 AND [1stDateofID] != [Date of Identification] THEN [Date of Identification] ELSE np.nan
df['1stDateofID'] = df['Client ID'].map(df.groupby('Client ID').agg({'Date of Identification':'min'})['Date of Identification'])
df['Return to Active Date'] = np.nan
df.loc[(df['Client ID Household Counter']>1) & (df['1stDateofID']!=df['Date of Identification'])
,"Return to Active Date"] = df['Date of Identification']
df.head()

,Client ID,Program Name,Veteran Status,Date of Identification,Program End Date,Dismissal Reason,Household Type,Client ID Counter,Client ID Household Counter,Chronic Status,Housing Move-In Date,Inactive Date,1stDateofID,Return to Active Date
0,903,Arlington Zero: Chronic - Veterans Only,No,2019/07/01 00:00:00,NaN,NaN,Single Adults,1,1.0,Yes,NaN,NaN,2019/07/01 00:00:00,NaN
1,3734,Arlington Zero: Single Adults,No,2021/04/09 00:00:00,NaN,NaN,Single Adults,1,1.0,NaN,NaN,NaN,2021/04/09 00:00:00,NaN
2,36862,Arlington Zero: Single Adults,No,2021/06/30 00:00:00,2022/02/04 00:00:00,Housed,Single Adults,1,1.0,NaN,2022/02/04 00:00:00,NaN,2021/06/30 00:00:00,NaN
3,42796,Arlington Zero: Single Adults,No,2021/09/08 00:00:00,NaN,NaN,Single Adults,1,1.0,NaN,NaN,NaN,2021/09/08 00:00:00,NaN
4,51829,Arlington Zero: Chronic - Veterans Only,Yes,2021/12/10 00:00:00,NaN,NaN,Single Adults,1,1.0,NaN,NaN,NaN,2021/12/10 00:00:00,NaN


In [203]:
df['Date of Identification'] = pd.to_datetime(df['Date of Identification'],format="%Y-%m-%d")
df['Program End Date'] = pd.to_datetime(df['Program End Date'],format="%Y-%m-%d")
df['Housing Move-In Date'] = pd.to_datetime(df['Housing Move-In Date'],format="%Y-%m-%d")
df['Inactive Date'] = pd.to_datetime(df['Inactive Date'],format="%Y-%m-%d")
df['1stDateofID'] = pd.to_datetime(df['1stDateofID'],format="%Y-%m-%d")
df['Return to Active Date'] = pd.to_datetime(df['Return to Active Date'],format="%Y-%m-%d")
df['Client ID Household Counter'] = df['Client ID Household Counter'].apply(int)
df.head()

,Client ID,Program Name,Veteran Status,Date of Identification,Program End Date,Dismissal Reason,Household Type,Client ID Counter,Client ID Household Counter,Chronic Status,Housing Move-In Date,Inactive Date,1stDateofID,Return to Active Date
0,903,Arlington Zero: Chronic - Veterans Only,No,2019-07-01,NaT,NaN,Single Adults,1,1,Yes,NaT,NaT,2019-07-01,NaT
1,3734,Arlington Zero: Single Adults,No,2021-04-09,NaT,NaN,Single Adults,1,1,NaN,NaT,NaT,2021-04-09,NaT
2,36862,Arlington Zero: Single Adults,No,2021-06-30,2022-02-04,Housed,Single Adults,1,1,NaN,2022-02-04,NaT,2021-06-30,NaT
3,42796,Arlington Zero: Single Adults,No,2021-09-08,NaT,NaN,Single Adults,1,1,NaN,NaT,NaT,2021-09-08,NaT
4,51829,Arlington Zero: Chronic - Veterans Only,Yes,2021-12-10,NaT,NaN,Single Adults,1,1,NaN,NaT,NaT,2021-12-10,NaT


In [204]:
#Step 9, Determine clients that "No longer meets population criteria" by demographic info
#All persons, all singles, veterans, chronic, chronic veteran, youth, families
print("How many clients this month No longer meet population criteria?")
print("All clients ",df['Dismissal Reason'].where(df['Dismissal Reason']=="No longer meets population criteria").count())
print("Singles ",df['Dismissal Reason'].where((df['Dismissal Reason']=="No longer meets population criteria")&(df['Household Type']=='Single Adults')).count())
print("Veterans ",df['Dismissal Reason'].where((df['Dismissal Reason']=="No longer meets population criteria")&(df['Veteran Status']=='Yes')).count())
print("Chronic ",df['Dismissal Reason'].where((df['Dismissal Reason']=="No longer meets population criteria")&(df['Chronic Status']=='Yes')).count())
print("Chronic Veterans ",df['Dismissal Reason'].where((df['Dismissal Reason']=="No longer meets population criteria")&(df['Chronic Status']=='Yes')&(df['Veteran Status']=='Yes')).count())
print("Youth ",df['Dismissal Reason'].where((df['Dismissal Reason']=="No longer meets population criteria")&(df['Household Type']=='Youth')).count())
print("Families ",df['Dismissal Reason'].where((df['Dismissal Reason']=="No longer meets population criteria")&(df['Household Type']=='Families')).count())


How many clients this month No longer meet population criteria?
All clients  4
Singles  4
Veterans  1
Chronic  2
Chronic Veterans  0
Youth  0
Families  0


In [205]:
#BFZ Reporting Metrics
print("Actively Homeless ", df['Client ID'].where(df['Program End Date'].isnull()==True).nunique())
print("Housing Placements ", df['Client ID'].where(df['Dismissal Reason']=='Housed').nunique())
print("Moved to Inactive ", df['Client ID'].where(df['Dismissal Reason']=='Inactive').nunique())
nii_counter = {}
for i in df['Client ID']:
    if i not in counter:
        nii_counter[i] = 0
for j in nii_counter:
    for k in df[df['Client ID']==j]['Date of Identification']:
        if k.year == 2022 and k.month == 2:
            nii_counter[i] += 1
        else:
            continue
# print("Newly Identified Inflow ", nii_counter)
df[df['Client ID']==2327]['Date of Identification']

Actively Homeless  38
Housing Placements  2
Moved to Inactive  0


18   2021-06-21
Name: Date of Identification, dtype: datetime64[ns]

In [206]:
df.dtypes

Client ID                               int64
Program Name                           object
Veteran Status                         object
Date of Identification         datetime64[ns]
Program End Date               datetime64[ns]
Dismissal Reason                       object
Household Type                         object
Client ID Counter                       int64
Client ID Household Counter             int64
Chronic Status                         object
Housing Move-In Date           datetime64[ns]
Inactive Date                  datetime64[ns]
1stDateofID                    datetime64[ns]
Return to Active Date          datetime64[ns]
dtype: object